## Group 8 Project
## Overview

This notebook was written to analyze Airbnb data.  Our members, alphabetically, are:
- Priya Choudhary
- Sophie Masud
- Sarah Otteson
- Manisha Sinha

Our goal is to estimate Airbnb rental prices based on a series of criteria such as whether the host is a superhost, the host identity is verified, how many amenities are included, etc.  Since price is continuous, we used the regression version of each algorythm.

At the end of the notebook, we will call out which model was most successful in estimating the Airbnb rental price (smallest Root Mean Square Error).

In [0]:
from pyspark.sql import SparkSession
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

spark = SparkSession.builder.appName('Group8RegressionModels').getOrCreate()

# File location and type
file_location = "/FileStore/tables/airbnb.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Renaming columns with missing underscore, remove columns that are categorical representations of the price column.  We will use price as our target variable
df = df.drop("price_gte_150","price_category").withColumnRenamed("Number of amenities","number_of_amenities")

df.show(5)
df.count()

+-----------------+----------------------+----------------------+-----------+------------+-------------+---------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|   latitude|   longitude|property_type|      room_type|accommodates|bathrooms|bedrooms|beds|bed_type|number_of_amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|
+-----------------+----------------------+----------------------+-----------+------------+-------------+---------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------

# Create a permanent table

In [0]:
# Commented out after initial run since the table already exists.
#permanent_table_name = "airbnb"
#df.write.format("parquet").mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
# Check the schema for the new df
df.printSchema()

root
 |-- host_is_superhost: integer (nullable = true)
 |-- host_identity_verified: integer (nullable = true)
 |-- neighbourhood_cleansed: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- property_type: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- beds: integer (nullable = true)
 |-- bed_type: string (nullable = true)
 |-- number_of_amenities: integer (nullable = true)
 |-- guests_included: integer (nullable = true)
 |-- price_per_extra_person: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- number_days_btw_first_last_review: integer (nullable = true)
 |-- review_scores_rating: integer (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- price: integer (nullable = true)



##Handle Null and Missing Data

In [0]:
from pyspark.sql.functions import col, when, count

df_Columns=["host_is_superhost","host_identity_verified","neighbourhood_cleansed","latitude","longitude","property_type","room_type","accommodates","bathrooms","bedrooms","beds","bed_type","number_of_amenities","guests_included","price_per_extra_person","minimum_nights",
            "number_of_reviews","number_days_btw_first_last_review","review_scores_rating","cancellation_policy","price"]

df_null_check = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() , c 
                           )).alias(c)
                    for c in df.columns])
df_null_check.show()

+-----------------+----------------------+----------------------+--------+---------+-------------+---------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|latitude|longitude|property_type|room_type|accommodates|bathrooms|bedrooms|beds|bed_type|number_of_amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|
+-----------------+----------------------+----------------------+--------+---------+-------------+---------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+
|               

In [0]:
import pyspark.sql.functions as F
df_null_count = df_null_check.withColumn('NULL_COUNT', sum([F.col(c) for c in df_Columns]))

print("Number of Rows with Nulls: ",df_null_count.first()['NULL_COUNT'])
print("Number of Total Rows: ",df.count())
print("Percent of Rows to Drop: ",round(df_null_count.first()['NULL_COUNT']/df.count()*100,2),"%")

# We will drop the nulls.  Most of the nulls are in "review_scores_rating" (800).  We think this would have an impact on the price and decided to keep this in the model


Number of Rows with Nulls:  836
Number of Total Rows:  3555
Percent of Rows to Drop:  23.52 %


In [0]:
df=df.dropna()

print("Number of Rows Remaining Post Null Drops: ",df.count())

Number of Rows Remaining Post Null Drops:  2732


## Look at the remaining data

In [0]:
df.show(5)

+-----------------+----------------------+----------------------+-----------+------------+-------------+------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|   latitude|   longitude|property_type|   room_type|accommodates|bathrooms|bedrooms|beds|bed_type|number_of_amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|
+-----------------+----------------------+----------------------+-----------+------------+-------------+------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+----------------

In [0]:
# review average price per property type
df.groupby('property_type').avg('price').show()

+---------------+------------------+
|  property_type|        avg(price)|
+---------------+------------------+
|      Apartment|170.43294237633862|
|      Townhouse|140.84444444444443|
|Bed & Breakfast|114.39285714285714|
|           Loft|             197.0|
|     Guesthouse|             289.0|
|          Villa|             140.0|
|   Entire Floor|              80.0|
|          Other|             198.0|
|           Dorm|              62.5|
|    Condominium|181.22988505747125|
|          House|123.53648068669528|
|           Boat| 277.3636363636364|
+---------------+------------------+



In [0]:
# Removing categorical and longitude/latitude columns to describe each column (count, mean, standard deviation, min, and max)
df_Numeric_Columns=["accommodates","bathrooms","bedrooms","beds","number_of_amenities","guests_included","price_per_extra_person","minimum_nights",
            "number_of_reviews","number_days_btw_first_last_review","review_scores_rating","price"]

# Get continuous columns' mean, stddev, min, and max
df_described=df.describe(df_Numeric_Columns)
display(df_described)


summary,accommodates,bathrooms,bedrooms,beds,number_of_amenities,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,price
count,2732,2732,2732,2732,2732,2732,2732,2732,2732,2732,2732,2732
mean,3.0439238653001466,1.2086383601756954,1.240117130307467,1.613103953147877,15.075036603221083,1.482796486090776,12.21303074670571,3.003294289897511,24.50475841874085,354.80563689604685,91.94363103953148,162.80124450951683
stddev,1.7941204613995216,0.4884518225712916,0.7486062143715101,1.0372150411325323,4.800739045705885,1.1029389577425122,19.766117054562155,8.942449348381423,38.54483728327186,424.3915677536139,9.517724589146676,100.56916945760833
min,1,0.0,0,0,0,0,0,1,1,0,20,11
max,16,6.0,5,16,30,14,200,300,404,2680,100,650


## Split Train/Test Data

In [0]:
#Break up the Data in Train (70%) and Test (30%) Datasets

train_data,test_data=df.randomSplit([0.70,0.30], seed=2023)

train_described=train_data.describe(df_Numeric_Columns)
display(train_described)

test_described=test_data.describe(df_Numeric_Columns)
display(test_described)

validation_check=df.count()*.3-test_data.count()
print("Validate split count: ",round(validation_check,0))



summary,accommodates,bathrooms,bedrooms,beds,number_of_amenities,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,price
count,1916,1916,1916,1916,1916,1916,1916,1916,1916,1916,1916,1916
mean,3.095511482254697,1.211116910229645,1.2562630480167014,1.644572025052192,15.071503131524008,1.5167014613778707,12.398225469728601,2.972338204592902,24.450417536534445,353.7886221294363,92.02139874739039,166.14039665970773
stddev,1.828055423448555,0.49717384565165623,0.7665237536914712,1.0783784241253547,4.823961302501493,1.1576661390134155,20.204091227581248,7.771123286945741,38.122834927461945,418.61479603946856,9.126100138306057,102.12712406156065
min,1,0.0,0,0,0,0,0,1,1,0,20,11
max,16,6.0,5,16,30,14,200,273,312,2636,100,650


summary,accommodates,bathrooms,bedrooms,beds,number_of_amenities,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,price
count,816,816,816,816,816,816,816,816,816,816,816,816
mean,2.922794117647059,1.2028186274509804,1.2022058823529411,1.5392156862745099,15.083333333333334,1.4031862745098038,11.778186274509803,3.075980392156863,24.63235294117647,357.1936274509804,91.76102941176471,154.9607843137255
stddev,1.7067660480676667,0.4675789584669724,0.703746354728446,0.9299020979138385,4.748694794308092,0.9582395128736022,18.702280026209536,11.22755643840321,39.54144685627411,437.9083460241684,10.382996998413926,96.41988151967175
min,1,0.0,0,0,0,0,0,1,1,0,20,29
max,16,6.0,5,9,27,8,150,300,404,2680,100,600


Validate split count:  4.0


In [0]:
from pyspark.ml.feature import VectorAssembler,StringIndexer,StandardScaler

In [0]:
# Convert categorical columns to numerical columns -- This will be used by each model 
neighbourhood_cleansed_indexer = StringIndexer(inputCol='neighbourhood_cleansed',outputCol='neighbourhood_cleansed_index',handleInvalid='keep')
property_type_indexer = StringIndexer(inputCol='property_type',outputCol='property_type_index',handleInvalid='keep')
room_type_indexer = StringIndexer(inputCol='room_type',outputCol='room_type_index',handleInvalid='keep')
bed_type_indexer = StringIndexer(inputCol='bed_type',outputCol='bed_type_index',handleInvalid='keep')
cancellation_policy_indexer = StringIndexer(inputCol='cancellation_policy',outputCol='cancellation_policy_index',handleInvalid='keep')


In [0]:
# Create the features -- This will be used by each model 
assembler = VectorAssembler(inputCols=['neighbourhood_cleansed_index','property_type_index','room_type_index','bed_type_index','cancellation_policy_index','host_is_superhost','host_identity_verified',
                                        'latitude','longitude','accommodates','bathrooms','bedrooms','beds','number_of_amenities','guests_included','price_per_extra_person','minimum_nights','number_of_reviews',
                                        'number_days_btw_first_last_review','review_scores_rating'], outputCol="features")



## Simple Linear Regression Model

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator 

In [0]:
#Model Definition
lr_model = LinearRegression(featuresCol='features',labelCol='price')

In [0]:
lr_pipe = Pipeline(stages=[neighbourhood_cleansed_indexer,property_type_indexer,room_type_indexer,bed_type_indexer,cancellation_policy_indexer,assembler,lr_model])

In [0]:
lr_fitted_model=lr_pipe.fit(train_data)

In [0]:
lr_results = lr_fitted_model.transform(test_data)

In [0]:
lr_results.select(['price','prediction']).show()

+-----+------------------+
|price|        prediction|
+-----+------------------+
|  103|  77.6762081707111|
|   88| 91.00593011432102|
|  120| 65.49044115385732|
|  150|153.00652683605585|
|   95| 121.5210352814629|
|  110| 136.7110233987123|
|  135|184.02012209036002|
|   52| 86.35234440199747|
|   60| 46.33053614975688|
|   60|32.570856676107724|
|  130| 173.1915866689069|
|  100|179.17730920228405|
|  160|138.14306312154622|
|  185|101.04234122075104|
|   97|121.82472414200492|
|  286| 248.3482511837774|
|  130|102.62273142887898|
|  239| 204.0689440852857|
|  165|135.02884630805966|
|  325|187.25117084229532|
+-----+------------------+
only showing top 20 rows



In [0]:
lr_results.show(3)

+-----------------+----------------------+----------------------+-----------+------------+-------------+------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+----------------------------+-------------------+---------------+--------------+-------------------------+--------------------+-----------------+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|   latitude|   longitude|property_type|   room_type|accommodates|bathrooms|bedrooms|beds|bed_type|number_of_amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|neighbourhood_cleansed_index|property_type_index|room_type_index|bed_type_index|cancellation_policy_index|            features|       prediction|
+-----------------+-----------------

In [0]:
lr_coef_df = spark.createDataFrame([{"inputColumnName": 'intercept',"Coefficient": str(lr_fitted_model.stages[-1].intercept),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[0])},
                       {"inputColumnName": 'neighbourhood_cleansed',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[0]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[1])},
                       {"inputColumnName": 'property_type',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[1]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[2])},
                       {"inputColumnName": 'room_type',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[2]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[3])},
                       {"inputColumnName": 'bed_type',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[3]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[4])},
                       {"inputColumnName": 'cancellation_policy',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[4]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[5])},
                       {"inputColumnName": 'host_is_superhost',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[5]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[6])},
                       {"inputColumnName": 'host_identity_verified',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[6]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[7])},
                       {"inputColumnName": 'latitude',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[7]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[8])},
                       {"inputColumnName": 'longitude',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[8]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[9])},
                       {"inputColumnName": 'accommodates',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[9]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[10])},
                       {"inputColumnName": 'bathrooms',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[10]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[11])},
                       {"inputColumnName": 'bedrooms',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[11]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[12])},
                       {"inputColumnName": 'beds',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[12]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[13])},
                       {"inputColumnName": 'number_of_amenities',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[13]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[14])},
                       {"inputColumnName": 'guests_included',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[14]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[15])},
                       {"inputColumnName": 'price_per_extra_person',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[15]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[16])},
                       {"inputColumnName": 'minimum_nights',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[16]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[17])},
                       {"inputColumnName": 'number_of_reviews',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[17]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[18])},
                       {"inputColumnName": 'number_days_btw_first_last_review',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[18]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[19])},
                       {"inputColumnName": 'review_scores_rating',"Coefficient": str(lr_fitted_model.stages[-1].coefficients[19]),"pValue":  str(lr_fitted_model.stages[-1].summary.pValues[20])}
                      ])

lr_coef_df.printSchema()
lr_coef_df.show(21)

root
 |-- Coefficient: string (nullable = true)
 |-- inputColumnName: string (nullable = true)
 |-- pValue: string (nullable = true)

+--------------------+--------------------+--------------------+
|         Coefficient|     inputColumnName|              pValue|
+--------------------+--------------------+--------------------+
|  -6058.280481976775|           intercept|  0.0041205357910159|
| -0.8302005522598946|neighbourhood_cle...|  0.4010619283685046|
|  1.2503721887320425|       property_type|                 0.0|
|  -71.24394982003281|           room_type|  0.1998298172651265|
|   4.638740919748304|            bed_type|0.000416223556670...|
|   6.451294455826183| cancellation_policy|0.000851061240249...|
|   15.52752301285984|   host_is_superhost|0.015993000865027085|
|   8.984576070193894|host_identity_ver...|                 0.0|
|   703.4243003770146|            latitude|5.694977822656710...|
|  333.54383900635804|           longitude|0.001391411748302...|
|  5.7520342401343765

In [0]:
lr_test_results_eval_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
lr_test_results_eval_rmse = lr_test_results_eval_rmse.evaluate(lr_results)
lr_test_results_eval_mae = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
lr_test_results_eval_mae = lr_test_results_eval_mae.evaluate(lr_results)
lr_test_results_eval_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
lr_test_results_eval_r2 = lr_test_results_eval_r2.evaluate(lr_results)
 
print("The Root Mean Square Error is ", round(lr_test_results_eval_rmse,2), ".  The Mean Absolute Error is ", round(lr_test_results_eval_mae,2), ". This model explains ", round(lr_test_results_eval_r2*100,2),"% of the variation of an Airbnb rental price.")

The Root Mean Square Error is  62.37 .  The Mean Absolute Error is  44.97 . This model explains  58.11 % of the variation of an Airbnb rental price.


## Decision Tree Regression Model

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor

In [0]:
dt_model =  DecisionTreeRegressor(featuresCol='features', labelCol='price')


In [0]:
# Process the indexer and assembler
dt_pipe = Pipeline(stages=[neighbourhood_cleansed_indexer,property_type_indexer,room_type_indexer,bed_type_indexer,cancellation_policy_indexer,assembler,dt_model])


In [0]:
dt_fitted_model = dt_pipe.fit(train_data)

In [0]:
dt_results = dt_fitted_model.transform(test_data)


In [0]:
dt_results.select(['price','prediction']).show()


+-----+------------------+
|price|        prediction|
+-----+------------------+
|  103| 74.81871345029239|
|   88| 74.81871345029239|
|  120| 62.02836879432624|
|  150|160.65068493150685|
|   95|           120.975|
|  110|160.65068493150685|
|  135|160.65068493150685|
|   52| 74.81871345029239|
|   60| 74.81871345029239|
|   60| 62.02836879432624|
|  130|160.65068493150685|
|  100|209.48701298701297|
|  160|209.48701298701297|
|  185|134.87755102040816|
|   97|134.87755102040816|
|  286|268.52941176470586|
|  130|134.87755102040816|
|  239|268.52941176470586|
|  165|162.82978723404256|
|  325|209.48701298701297|
+-----+------------------+
only showing top 20 rows



In [0]:
# Show 3 rows of preduction with all columns and features (truncated, too many columns)
dt_results.show(3)

+-----------------+----------------------+----------------------+-----------+------------+-------------+------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+----------------------------+-------------------+---------------+--------------+-------------------------+--------------------+-----------------+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|   latitude|   longitude|property_type|   room_type|accommodates|bathrooms|bedrooms|beds|bed_type|number_of_amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|neighbourhood_cleansed_index|property_type_index|room_type_index|bed_type_index|cancellation_policy_index|            features|       prediction|
+-----------------+-----------------

In [0]:
# Print the tree in string format (values are more readable, but it is a bit hard to follow with a deep tree.  Added the columns to map to the feature #s displayed in the tree)
index=assembler.getInputCols()
print(index)
print(dt_fitted_model.stages[-1].toDebugString)


['neighbourhood_cleansed_index', 'property_type_index', 'room_type_index', 'bed_type_index', 'cancellation_policy_index', 'host_is_superhost', 'host_identity_verified', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_amenities', 'guests_included', 'price_per_extra_person', 'minimum_nights', 'number_of_reviews', 'number_days_btw_first_last_review', 'review_scores_rating']
DecisionTreeRegressionModel: uid=DecisionTreeRegressor_95f6ad7a6e72, depth=5, numNodes=61, numFeatures=20
  If (feature 2 in {1.0,2.0})
   If (feature 0 in {0.0,3.0,4.0,6.0,8.0,11.0,12.0,15.0,16.0,18.0,20.0,22.0,23.0})
    If (feature 14 <= 3.5)
     If (feature 9 <= 1.5)
      If (feature 3 in {2.0})
       Predict: 34.111111111111114
      Else (feature 3 not in {2.0})
       Predict: 62.02836879432624
     Else (feature 9 > 1.5)
      If (feature 0 in {3.0,6.0,16.0,18.0,20.0,22.0})
       Predict: 74.81871345029239
      Else (feature 0 not in {3.0,6.0,16.0,18.0,20.0,22.0})
     

In [0]:
# Display the tree in tree form.  Without pyspark_tree_print access, making it more readable did not seem reachable. Added the columns to map to the feature #s displayed in the tree)
index=assembler.getInputCols()
print(index)
dt_tree = dt_fitted_model.stages[-1]
display(dt_tree)

['neighbourhood_cleansed_index', 'property_type_index', 'room_type_index', 'bed_type_index', 'cancellation_policy_index', 'host_is_superhost', 'host_identity_verified', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_amenities', 'guests_included', 'price_per_extra_person', 'minimum_nights', 'number_of_reviews', 'number_days_btw_first_last_review', 'review_scores_rating']


treeNode
"{""index"":29,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[1.0,2.0],""feature"":2,""overflow"":false}"
"{""index"":13,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[0.0,3.0,4.0,6.0,8.0,11.0,12.0,15.0,16.0,18.0,20.0,22.0,23.0],""feature"":0,""overflow"":false}"
"{""index"":7,""featureType"":""continuous"",""prediction"":null,""threshold"":3.5,""categories"":null,""feature"":14,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":1.5,""categories"":null,""feature"":9,""overflow"":false}"
"{""index"":1,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[2.0],""feature"":3,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":34.111111111111114,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":62.02836879432624,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":5,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[3.0,6.0,16.0,18.0,20.0,22.0],""feature"":0,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":74.81871345029239,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":85.9139344262295,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [0]:
dt_test_results_eval_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
dt_test_results_eval_rmse = dt_test_results_eval_rmse.evaluate(dt_results)
dt_test_results_eval_mae = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
dt_test_results_eval_mae = dt_test_results_eval_mae.evaluate(dt_results)
dt_test_results_eval_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
dt_test_results_eval_r2 = dt_test_results_eval_r2.evaluate(dt_results)

print("The Root Mean Square Error is ", round(dt_test_results_eval_rmse,2), ".  The Mean Absolute Error is ", round(dt_test_results_eval_mae,2), ". This model explains ", round(dt_test_results_eval_r2*100,2),"% of the variation of an Airbnb rental price.")


The Root Mean Square Error is  63.56 .  The Mean Absolute Error is  41.34 . This model explains  56.5 % of the variation of an Airbnb rental price.


## Random Forest Tree Regression Model

In [0]:
from pyspark.ml.regression import RandomForestRegressor

In [0]:
# Train a RandomForest model.
rf_model = RandomForestRegressor(featuresCol='features', labelCol='price')


In [0]:
# Process the indexer and assembler
RandomForestPipe = Pipeline(stages=[neighbourhood_cleansed_indexer,property_type_indexer,room_type_indexer,bed_type_indexer,cancellation_policy_indexer, assembler,rf_model])


In [0]:
RandomForest_Fitted_Model = RandomForestPipe.fit(train_data)

In [0]:
RFD_Results = RandomForest_Fitted_Model.transform(test_data)

In [0]:
RFD_Results.select(['price','prediction']).show()

+-----+------------------+
|price|        prediction|
+-----+------------------+
|  103| 79.70630364539397|
|   88| 82.57445126744786|
|  120| 72.48754762255616|
|  150|127.55416476929697|
|   95|114.05094849989257|
|  110| 123.1144946901389|
|  135|139.13941825443038|
|   52|  80.3355814205352|
|   60|  77.6663411829129|
|   60| 66.21890410563152|
|  130|132.71616951982335|
|  100|201.07275733406385|
|  160|181.59292060733216|
|  185|127.99675157997449|
|   97|127.99675157997449|
|  286|239.89785530519845|
|  130|131.74762546309546|
|  239| 234.9022960336253|
|  165| 172.9357459580139|
|  325|195.49812242144975|
+-----+------------------+
only showing top 20 rows



In [0]:
# Show 3 rows of preduction with all columns and features (truncated, too many columns)
RFD_Results.show(3)

+-----------------+----------------------+----------------------+-----------+------------+-------------+------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+----------------------------+-------------------+---------------+--------------+-------------------------+--------------------+-----------------+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|   latitude|   longitude|property_type|   room_type|accommodates|bathrooms|bedrooms|beds|bed_type|number_of_amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|neighbourhood_cleansed_index|property_type_index|room_type_index|bed_type_index|cancellation_policy_index|            features|       prediction|
+-----------------+-----------------

In [0]:
# Print the tree in string format (values are more readable, but it is a bit hard to follow with a deep tree.  Added the columns to map to the feature #s displayed in the tree)
index=assembler.getInputCols()
print(index)
print(RandomForest_Fitted_Model.stages[-1].toDebugString)


['neighbourhood_cleansed_index', 'property_type_index', 'room_type_index', 'bed_type_index', 'cancellation_policy_index', 'host_is_superhost', 'host_identity_verified', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_amenities', 'guests_included', 'price_per_extra_person', 'minimum_nights', 'number_of_reviews', 'number_days_btw_first_last_review', 'review_scores_rating']
RandomForestRegressionModel: uid=RandomForestRegressor_cf625f73c3b0, numTrees=20, numFeatures=20
  Tree 0 (weight 1.0):
    If (feature 11 <= 1.5)
     If (feature 7 <= 42.331950160000005)
      If (feature 14 <= 1.5)
       If (feature 2 in {1.0,2.0})
        If (feature 0 in {3.0,15.0,16.0,18.0,20.0,22.0})
         Predict: 67.21637426900585
        Else (feature 0 not in {3.0,15.0,16.0,18.0,20.0,22.0})
         Predict: 82.63265306122449
       Else (feature 2 not in {1.0,2.0})
        If (feature 17 <= 2.5)
         Predict: 145.66666666666666
        Else (feature 17 > 2.5)
   

In [0]:
RFD_test_results_eval_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
RFD_test_results_eval_rmse = RFD_test_results_eval_rmse.evaluate(RFD_Results)
RFD_test_results_eval_mae = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
RFD_test_results_eval_mae = RFD_test_results_eval_mae.evaluate(RFD_Results)
RFD_test_results_eval_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
RFD_test_results_eval_r2 = RFD_test_results_eval_r2.evaluate(RFD_Results)
 
print("The Root Mean Square Error is ", round(RFD_test_results_eval_rmse,2), ".  The Mean Absolute Error is ", round(RFD_test_results_eval_mae,2), ". This model explains ", round(RFD_test_results_eval_r2*100,2),"% of the variation of an Airbnb rental price.")
 


The Root Mean Square Error is  54.56 .  The Mean Absolute Error is  37.71 . This model explains  67.94 % of the variation of an Airbnb rental price.


## Gradient-Boosted Tree Regression Model

In [0]:
from pyspark.ml.regression import GBTRegressor

In [0]:
gbtr_model =  GBTRegressor(featuresCol='features', labelCol='price', maxIter=10)

In [0]:
# Process the indexer and assembler
gbtr_pipe = Pipeline(stages=[neighbourhood_cleansed_indexer,property_type_indexer,room_type_indexer,bed_type_indexer,cancellation_policy_indexer,assembler,gbtr_model])


In [0]:
gbtr_fitted_model = gbtr_pipe.fit(train_data)

In [0]:
gbtr_results = gbtr_fitted_model.transform(test_data)


In [0]:
gbtr_results.select(['price','prediction']).show()


+-----+------------------+
|price|        prediction|
+-----+------------------+
|  103| 69.73161583621703|
|   88|  71.4979864315359|
|  120|58.545634609546305|
|  150|148.60714520979624|
|   95|112.10042905361182|
|  110|146.84077461447734|
|  135|153.48731211358927|
|   52| 69.63266804440366|
|   60|  71.4979864315359|
|   60| 54.88208872197479|
|  130|146.84077461447734|
|  100|202.76097141658082|
|  160| 206.7013337708396|
|  185| 135.2916103898067|
|   97|148.55828388531822|
|  286| 300.8850937286913|
|  130|129.20823104435235|
|  239| 265.8739083416663|
|  165|160.04410801786918|
|  325|208.90688327264897|
+-----+------------------+
only showing top 20 rows



In [0]:
# Show 3 rows of preduction with all columns and features (truncated, too many columns)
gbtr_results.show(3)

+-----------------+----------------------+----------------------+-----------+------------+-------------+------------+------------+---------+--------+----+--------+-------------------+---------------+----------------------+--------------+-----------------+---------------------------------+--------------------+-------------------+-----+----------------------------+-------------------+---------------+--------------+-------------------------+--------------------+------------------+
|host_is_superhost|host_identity_verified|neighbourhood_cleansed|   latitude|   longitude|property_type|   room_type|accommodates|bathrooms|bedrooms|beds|bed_type|number_of_amenities|guests_included|price_per_extra_person|minimum_nights|number_of_reviews|number_days_btw_first_last_review|review_scores_rating|cancellation_policy|price|neighbourhood_cleansed_index|property_type_index|room_type_index|bed_type_index|cancellation_policy_index|            features|        prediction|
+-----------------+---------------

In [0]:
# Print the tree in string format (values are more readable, but it is a bit hard to follow with a deep tree.  Added the columns to map to the feature #s displayed in the tree)
index=assembler.getInputCols()
print(index)
print(gbtr_fitted_model.stages[-1].toDebugString)


['neighbourhood_cleansed_index', 'property_type_index', 'room_type_index', 'bed_type_index', 'cancellation_policy_index', 'host_is_superhost', 'host_identity_verified', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_amenities', 'guests_included', 'price_per_extra_person', 'minimum_nights', 'number_of_reviews', 'number_days_btw_first_last_review', 'review_scores_rating']
GBTRegressionModel: uid=GBTRegressor_f0f16e1aeedc, numTrees=10, numFeatures=20
  Tree 0 (weight 1.0):
    If (feature 2 in {1.0,2.0})
     If (feature 0 in {0.0,3.0,4.0,6.0,8.0,11.0,12.0,15.0,16.0,18.0,20.0,22.0,23.0})
      If (feature 14 <= 3.5)
       If (feature 9 <= 1.5)
        If (feature 3 in {2.0})
         Predict: 34.111111111111114
        Else (feature 3 not in {2.0})
         Predict: 62.02836879432624
       Else (feature 9 > 1.5)
        If (feature 0 in {3.0,6.0,16.0,18.0,20.0,22.0})
         Predict: 74.81871345029239
        Else (feature 0 not in {3.0,6.0,16.0,18

In [0]:
gbtr_test_results_eval_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
gbtr_test_results_eval_rmse = gbtr_test_results_eval_rmse.evaluate(gbtr_results)
gbtr_test_results_eval_mae = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
gbtr_test_results_eval_mae = gbtr_test_results_eval_mae.evaluate(gbtr_results)
gbtr_test_results_eval_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
gbtr_test_results_eval_r2 = gbtr_test_results_eval_r2.evaluate(gbtr_results)

print("The Root Mean Square Error is ", round(gbtr_test_results_eval_rmse,2), ".  The Mean Absolute Error is ", round(gbtr_test_results_eval_mae,2), ". This model explains ", round(gbtr_test_results_eval_r2*100,2),"% of the variation of an Airbnb rental price.")


The Root Mean Square Error is  61.01 .  The Mean Absolute Error is  38.5 . This model explains  59.91 % of the variation of an Airbnb rental price.


## Model Comparison

In [0]:
from pyspark.sql.functions import min
from pyspark.sql import Row

model_comparison_df = spark.createDataFrame([{"modelName": 'Simple Logistic Regression', "rootMeanSquareError": round(lr_test_results_eval_rmse,2), "meanAbsoluteError": round(lr_test_results_eval_mae,2), "r2Percent": round(lr_test_results_eval_r2*100,2)},
                       {"modelName": 'Decision Tree Regression', "rootMeanSquareError": round(dt_test_results_eval_rmse,2), "meanAbsoluteError": round(dt_test_results_eval_mae,2), "r2Percent": round(dt_test_results_eval_r2*100,2)},
                       {"modelName": 'Random Forest Tree Regression', "rootMeanSquareError": round(RFD_test_results_eval_rmse,2), "meanAbsoluteError": round(RFD_test_results_eval_mae,2), "r2Percent": round(RFD_test_results_eval_r2*100,2)},
                       {"modelName": 'Gradient-Boosted Tree Regression', "rootMeanSquareError": round(gbtr_test_results_eval_rmse,2), "meanAbsoluteError": round(gbtr_test_results_eval_mae,2), "r2Percent": round(gbtr_test_results_eval_r2*100,2)}
                      ])
model_comparison_df.show()

+-----------------+--------------------+---------+-------------------+
|meanAbsoluteError|           modelName|r2Percent|rootMeanSquareError|
+-----------------+--------------------+---------+-------------------+
|            44.97|Simple Logistic R...|    58.11|              62.37|
|            41.34|Decision Tree Reg...|     56.5|              63.56|
|            37.71|Random Forest Tre...|    67.94|              54.56|
|             38.5|Gradient-Boosted ...|    59.91|              61.01|
+-----------------+--------------------+---------+-------------------+



In [0]:
min_rmse = model_comparison_df.agg({"rootMeanSquareError": "min"}).collect()[0][0]

lowest_df_row = model_comparison_df.filter(model_comparison_df.rootMeanSquareError <= min_rmse)
lowest_df_row=lowest_df_row.collect()

print("The best algorithm is the ", lowest_df_row[0][1], ".  For this algorithm and model, the Root Mean Square Error is ", lowest_df_row[0][3], " and the Mean Absolute Error is ", lowest_df_row[0][0], ".  It explains ", lowest_df_row[0][2], "% of the variation of an Airbnb rental price.")



The best algorithm is the  Random Forest Tree Regression .  For this algorithm and model, the Root Mean Square Error is  54.56  and the Mean Absolute Error is  37.71 .  It explains  67.94 % of the variation of an Airbnb rental price.
